# Winter 2022 and 2023 Precipitation and Wind Characteristics

Author: Daniel Hogan
Created: January 17, 2024

This notebook will look at wind speed, direction, and precipitation to deduce how winds and precipitatiopn are related. This will help provide evidence in support (or against) my hypothesis that less snow means less sublimation, even if it is windy.

### Imports

In [1]:
# general
import os
import glob
import datetime as dt
import json
# data 
import xarray as xr 
from sublimpy import utils, variables, tidy, turbulence
import numpy as np
import pandas as pd
from act import discovery, plotting
# plotting
import matplotlib.pyplot as plt
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# helper tools
from scripts.get_sail_data import get_sail_data
from scripts.helper_funcs import create_windrose_df, simple_sounding, mean_sounding
import scripts.helper_funcs as hf
from metpy import calc, units
import time
# make plotly work 
init_notebook_mode(connected=True)
cf.go_offline()

nctoolkit is using Climate Data Operators version 2.3.0


# Laser Disdrometer data
Here, I will gather the laser disdrometer data and organize it into hourly data to plot alongside wind speeds and direction.

In [59]:
# gather laser disdrometer data for 2023
w23_ld_mtcb_ds = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_22_23/laser_disdrometer_mt_cb_20221001_20230930.nc')
w23_ld_gts_ds = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_22_23/laser_disdrometer_gothic_20221001_20230930.nc')

# gather laser disdrometer data for 2022
w22_ld_mtcb_ds = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_21_22/laser_disdrometer_mt_cb_20211001_20220930.nc')
w22_ld_gts_ds = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_21_22/laser_disdrometer_gothic_20211001_20220930.nc')

Laser disdrometer is 1 minute data, with other dimensions of particle size (mm) and raw_fall_velocity (m/s).
The variable of interest is `precip_rate` and `qc_precip_rate`

In [60]:
# let's look at some of the data
w23_ld_mtcb_ds

<xarray.Dataset>
Dimensions:                               (time: 372545, particle_size: 32,
                                           raw_fall_velocity: 32)
Coordinates:
  * time                                  (time) datetime64[ns] 2022-10-01 .....
  * particle_size                         (particle_size) float32 0.062 ... 24.0
  * raw_fall_velocity                     (raw_fall_velocity) float32 0.05 .....
Data variables: (12/42)
    base_time                             (time) datetime64[ns] ...
    time_offset                           (time) datetime64[ns] ...
    precip_rate                           (time) float32 ...
    qc_precip_rate                        (time) int32 ...
    weather_code                          (time) float32 ...
    qc_weather_code                       (time) int32 ...
    ...                                    ...
    moment4                               (time) float32 ...
    moment5                               (time) float32 ...
    moment6                               (time) float32 ...
    lat                                   (time) float32 ...
    lon                                   (time) float32 ...
    alt                                   (time) float32 ...
Attributes: (12/15)
    command_line:                ld_ingest -s guc -f S2
    process_version:             ingest-ld-1.4-0.el7
    dod_version:                 ld-b1-2.0
    Conventions:                 ARM-1.2
    data_level:                  b1
    platform_id:                 ld
    ...                          ...
    facility_id:                 S2
    sampling_interval:           1 minute
    effective_measurement_area:  0.54 m^2
    doi:                         10.5439/1560674
    datastream:                  gucldS2.b1
    history:                     created by user dsmgr on machine prod-proc4....

In [61]:
# Now let's filtter the data to our winter period (Dec 1 - Mar 31) and only look at the precip_rate data
w23_prcp_rate_mtcb_ds = w23_ld_mtcb_ds.sel(time=slice('2022-12-01', '2023-03-31'))[['precip_rate', 'qc_precip_rate']]
w23_prcp_rate_gts_ds = w23_ld_gts_ds.sel(time=slice('2022-12-01', '2023-03-31'))[['precip_rate', 'qc_precip_rate']]
w22_prcp_rate_mtcb_ds = w22_ld_mtcb_ds.sel(time=slice('2021-12-01', '2022-03-31'))[['precip_rate', 'qc_precip_rate']]
w22_prcp_rate_gts_ds = w22_ld_gts_ds.sel(time=slice('2021-12-01', '2022-03-31'))[['precip_rate', 'qc_precip_rate']]

In [62]:
# Now let's filter any data that has a qc_precip_rate value of 0
w23_prcp_rate_mtcb_ds = w23_prcp_rate_mtcb_ds.where(w23_prcp_rate_mtcb_ds.qc_precip_rate == 0, np.nan)
w23_prcp_rate_gts_ds = w23_prcp_rate_gts_ds.where(w23_prcp_rate_gts_ds.qc_precip_rate == 0, np.nan)
w22_prcp_rate_mtcb_ds = w22_prcp_rate_mtcb_ds.where(w22_prcp_rate_mtcb_ds.qc_precip_rate == 0, np.nan)
w22_prcp_rate_gts_ds = w22_prcp_rate_gts_ds.where(w22_prcp_rate_gts_ds.qc_precip_rate == 0, np.nan)
# now let's resample the data to hourly sums
w23_prcp_rate_1H_mtcb_ds = w23_prcp_rate_mtcb_ds.resample(time='1H').sum()/60
w23_prcp_rate_1H_gts_ds = w23_prcp_rate_gts_ds.resample(time='1H').sum()/60
w22_prcp_rate_1H_mtcb_ds = w22_prcp_rate_mtcb_ds.resample(time='1H').sum()/60
w22_prcp_rate_1H_gts_ds = w22_prcp_rate_gts_ds.resample(time='1H').sum()/60

2022

In [63]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=w22_prcp_rate_1H_mtcb_ds.time,
                         y=w22_prcp_rate_1H_mtcb_ds.precip_rate,
                         mode='lines',
                         marker=dict(size=5),
                         name='Mt. Crested Butte'))
fig.add_trace(go.Scatter(x=w22_prcp_rate_1H_gts_ds.time,
                         y=w22_prcp_rate_1H_gts_ds.precip_rate,
                         mode='lines',
                         marker=dict(size=5),
                         name='Gothic Townsite'))
fig.update_layout(title='Winter Precipitation (Dec 1 - Mar 31) 2021-2022',
                    xaxis_title='Time',
                    yaxis_title='Precipitation Rate (mm/hr)',
                    height=600,
                    width=800)

# add a second y-axis
fig.update_layout(yaxis2=dict(title='Cumulative Precipitation (mm)',
                              overlaying='y',
                              side='right'))
fig.add_trace(go.Scatter(x=w22_prcp_rate_1H_mtcb_ds.time,
                            y=w22_prcp_rate_1H_mtcb_ds.precip_rate.cumsum(),
                            mode='lines',
                            marker=dict(size=5,
                                        color='blue'),
                            name='Mt. Crested Butte',
                            showlegend=False,
                            yaxis='y2'))
fig.add_trace(go.Scatter(x=w22_prcp_rate_1H_gts_ds.time,
                            y=w22_prcp_rate_1H_gts_ds.precip_rate.cumsum(),
                            mode='lines',
                            marker=dict(size=5,
                                        color='red'),
                            name='Gothic Townsite',
                            showlegend=False,
                            yaxis='y2'))
# move the legend up a bit
fig.update_layout(legend=dict(y=1.2))

2023

In [64]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=w23_prcp_rate_1H_mtcb_ds.time,
                         y=w23_prcp_rate_1H_mtcb_ds.precip_rate,
                         mode='lines',
                         marker=dict(size=5),
                         name='Mt. Crested Butte'))
fig.add_trace(go.Scatter(x=w23_prcp_rate_1H_gts_ds.time,
                         y=w23_prcp_rate_1H_gts_ds.precip_rate,
                         mode='lines',
                         marker=dict(size=5),
                         name='Gothic Townsite'))
fig.update_layout(title='Winter Precipitation (Dec 1 - Mar 31) 2022-2023',
                    xaxis_title='Time',
                    yaxis_title='Precipitation Rate (mm/hr)',
                    height=600,
                    width=800)

# add a second y-axis
fig.update_layout(yaxis2=dict(title='Cumulative Precipitation (mm)',
                              overlaying='y',
                              side='right'))
fig.add_trace(go.Scatter(x=w23_prcp_rate_1H_mtcb_ds.time,
                            y=w23_prcp_rate_1H_mtcb_ds.precip_rate.cumsum(),
                            mode='lines',
                            marker=dict(size=5,
                                        color='blue'),
                            name='Mt. Crested Butte',
                            showlegend=False,
                            yaxis='y2'))
fig.add_trace(go.Scatter(x=w23_prcp_rate_1H_gts_ds.time,
                            y=w23_prcp_rate_1H_gts_ds.precip_rate.cumsum(),
                            mode='lines',
                            marker=dict(size=5,
                                        color='red'),
                            name='Gothic Townsite',
                            showlegend=False,
                            yaxis='y2'))
# move the legend up a bit
fig.update_layout(legend=dict(y=1.2))

### Wind Data

In [189]:
# Now let's grab wind data from SOS for 2023 at Kettle Ponds and wind data from SAIL at Gothic
sos_1H_max_wspd_ds = xr.open_dataset('../../01_data/processed_data/sos_ds_1H_max_wspd_storage.nc')
sos_1H_mean_wspd_ds = xr.open_dataset('../../01_data/raw_data/sos_ds_1H_storage.nc')
# SOS data is in utc, let's convert it to mountain time
# First let's add the UTC timezone to the datasets 
sos_1H_max_wspd_ds['time'] = pd.to_datetime(sos_1H_max_wspd_ds.time).tz_localize('UTC').tz_convert('MST').tz_localize(None)
sos_1H_mean_wspd_ds['time'] = pd.to_datetime(sos_1H_mean_wspd_ds.time).tz_localize('UTC').tz_convert('MST').tz_localize(None)


# now let's grab wind data from SAIL in Gothic 
w22_sail_met_qc_ds_1H = xr.open_dataset('../../01_data/processed_data/sail_processed/sail_ds_1H_met_2022.nc')
w23_sail_met_qc_ds_1H = xr.open_dataset('../../01_data/processed_data/sail_processed/sail_ds_1H_met_2023.nc')

In [198]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=w23_prcp_rate_1H_gts_ds.time,
                         y=w23_prcp_rate_1H_gts_ds.precip_rate,
                         mode='lines',
                         marker=dict(size=5),
                         name='Gothic Townsite'))
fig.update_layout(title='Winter Precipitation (Dec 1 - Mar 31) 2022-2023',
                    xaxis_title='Time',
                    yaxis_title='Precipitation Rate (mm/hr)',
                    # height=600,
                    # width=800
)

# add a second y-axis
fig.update_layout(yaxis2=dict(title='Wind Gust (m/s)',
                              overlaying='y',
                              side='right'))
fig.add_trace(go.Scatter(x=w23_sail_met_qc_ds_1H.time,
                            y=w23_sail_met_qc_ds_1H.wspd_arith_mean,
                            mode='lines',
                            marker=dict(size=5,
                                        color='red'),
                            name='10-m wind gust at Gothic',
                            showlegend=False,
                            yaxis='y2'))
# move the legend up a bit
fig.update_layout(legend=dict(y=1.2))

There appears to be a rise in wind gust speed during stormy periods, but the signal to noise ratio is very low, so not much can be deduced from this

In [199]:
# let's make a scatter plot next 
fig = go.Figure()
fig.add_trace(go.Scatter(x=w23_prcp_rate_1H_gts_ds.precip_rate,
                         y=w23_sail_met_qc_ds_1H.sel(time=w23_prcp_rate_1H_gts_ds.time).wspd_arith_mean,
                         mode='markers',
                         marker=dict(size=5),
                         ))
fig.update_layout(title='Precipitation Rate vs. 10-m Wind Gust',
                    xaxis_title='Precipitation Rate (mm/hr)',
                    yaxis_title='10-m Wind Gust (m/s)',
                    height=600,
                    width=800
)


Not much here, but we can generally say, when it is snowing, it is often also windy. What about looking at how these variables covary at different lag times?

In [200]:
# let's add these two dataset together into a dataframe
w23_prcp_rate_1H_gts_df = w23_prcp_rate_1H_gts_ds.to_dataframe()
w22_prcp_rate_1H_gts_df = w22_prcp_rate_1H_gts_ds.to_dataframe()
# create a binary precipitation dataset for 2022
binary_22_df = w22_prcp_rate_1H_gts_df['precip_rate'].apply(lambda x: 1 if x > 0 else 0)
binary_22_df.name = 'precip_binary'
# create a binary precipitation dataset for 2023
binary_23_df = w23_prcp_rate_1H_gts_df['precip_rate'].apply(lambda x: 1 if x > 0 else 0)
binary_23_df.name = 'precip_binary'


sos_1H_mean_wspd_df = sos_1H_mean_wspd_ds.sel(time=w23_prcp_rate_1H_gts_ds.time).to_dataframe()
sail_1H_mean_wspd_22_df = w22_sail_met_qc_ds_1H.to_dataframe()
sail_1H_mean_wspd_23_df = w23_sail_met_qc_ds_1H.to_dataframe()

combined_sos_sail_df = pd.concat([binary_23_df, sos_1H_mean_wspd_df], axis=1)[['precip_binary','spd_10m_uw']]
combined_sail_23_df = pd.concat([binary_23_df, sail_1H_mean_wspd_23_df], axis=1)[['precip_binary','wspd_arith_mean']]
combined_sail_22_df = pd.concat([binary_22_df, sail_1H_mean_wspd_22_df], axis=1)[['precip_binary','wspd_arith_mean']]

Check for stationarity

In [201]:
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    result = pd.Series(dftest[0:4], index=['Test Statistic','P-value','Lags Used','No of Observations'])
    for key,value in dftest[4].items():
        result['Critical Value (%s)'%key] = value
    return result
    
# adf_table = df.drop('week', axis = 1)
df = combined_sail_22_df.dropna().apply(adf_test, axis = 0)
df

,precip_binary,wspd_arith_mean
Test Statistic,-7.349540e+00,-7.018828e+00
P-value,1.014506e-10,6.624279e-10
Lags Used,1.300000e+01,2.700000e+01
No of Observations,2.885000e+03,2.871000e+03
Critical Value (1%),-3.432619e+00,-3.432630e+00
Critical Value (5%),-2.862542e+00,-2.862547e+00
Critical Value (10%),-2.567304e+00,-2.567306e+00


Cross-correlation for 2022 between SAIL wind and SAIL Precipitation

In [202]:
# Calculate cross-correlation
max_lag = 24
lags = np.arange(-max_lag-1, max_lag + 1)
cross_corr = [combined_sail_22_df['wspd_arith_mean'].corr(combined_sail_22_df['precip_binary'].shift(-lag)) for lag in lags]
cross_corr
# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=lags, y=cross_corr, mode='lines+markers', name='Cross-correlation'))

fig.update_layout(
    title='Time-Lagged Cross-Correlation between Wind Speed and Precipitation' ,
    xaxis_title='Lag (hours)',
    yaxis_title='Cross-Correlation',
    xaxis=dict(range=[-24.25, 24.25],
               tickmode='linear',
               tickfont=dict(size=18)),
    yaxis=dict(range=[-0.4, 0.4], tickfont=dict(size=20)),
    width=1000,
    height=600
)
# increase yaxis font label size
fig.update_yaxes(title_font=dict(size=20))
fig.update_xaxes(title_font=dict(size=18))

fig.show()


Cross-correlation for 2023 between SAIL wind and SAIL Precipitation

In [203]:
# Calculate cross-correlation
max_lag = 24
lags = np.arange(-max_lag-1, max_lag + 1)
cross_corr = [combined_sail_23_df['wspd_arith_mean'].corr(combined_sail_23_df['precip_binary'].shift(-lag)) for lag in lags]
cross_corr
# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=lags, y=cross_corr, mode='lines+markers', name='Cross-correlation'))

fig.update_layout(
    title='Time-Lagged Cross-Correlation between Wind Speed and Precipitation' ,
    xaxis_title='Lag (hours)',
    yaxis_title='Cross-Correlation',
    xaxis=dict(range=[-24.25, 24.25],
               tickmode='linear',
               tickfont=dict(size=18)),
    yaxis=dict(range=[-0.4, 0.4], tickfont=dict(size=20)),
    width=1000,
    height=600
)
# increase yaxis font label size
fig.update_yaxes(title_font=dict(size=20))
fig.update_xaxes(title_font=dict(size=18))

fig.show()


Cross-correlation for 2023 between SOS wind and SAIL Precipitation

In [204]:
# Calculate cross-correlation
max_lag = 24
lags = np.arange(-max_lag-1, max_lag + 1)
cross_corr = [combined_sos_sail_df['spd_10m_uw'].corr(combined_sos_sail_df['precip_binary'].shift(-lag)) for lag in lags]
cross_corr
# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=lags, y=cross_corr, mode='lines+markers', name='Cross-correlation'))

fig.update_layout(
    title='Time-Lagged Cross-Correlation between Wind Speed and Precipitation' ,
    xaxis_title='Lag (hours)',
    yaxis_title='Cross-Correlation',
    xaxis=dict(range=[-24.25, 24.25],
               tickmode='linear',
               tickfont=dict(size=18)),
    yaxis=dict(range=[-0.4, 0.4], tickfont=dict(size=20)),
    width=1000,
    height=600
)
# increase yaxis font label size
fig.update_yaxes(title_font=dict(size=20))
fig.update_xaxes(title_font=dict(size=18))

fig.show()


Cop

In [178]:
fn_2022 = "/storage/elilouis/sublimationofsnow/tilden_precip_data/KP22_274-365.PRECIP1"
fn_2023 = "/storage/elilouis/sublimationofsnow/tilden_precip_data/KP23_001-177.PRECIP1"

df = pd.concat([
    pd.read_csv(fn_2022, delim_whitespace=True),
    pd.read_csv(fn_2023, delim_whitespace=True)
])
# create timestamps from strings
df['date'] = pd.to_datetime(df['date'])
# calculate accumulated precip
# combine data and time(MST) columns into a single datetime column
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time(MST)'].astype(str))
df

,date,time(MST),DecDay,Day,Precip_mm,Geonor1,Geonor2,Geonor3,TPRECP_Max,datetime
0,2022-10-01,00:30:00,274.021,274.0,0.0,0.0,0.0,0.0,-56.94,2022-10-01 00:30:00
1,2022-10-01,01:00:00,274.042,274.0,0.0,0.0,0.0,0.0,-56.93,2022-10-01 01:00:00
2,2022-10-01,01:30:00,274.062,274.0,0.0,0.0,0.0,0.0,-56.94,2022-10-01 01:30:00
3,2022-10-01,02:00:00,274.083,274.0,0.0,0.0,0.0,0.0,-56.94,2022-10-01 02:00:00
4,2022-10-01,02:30:00,274.104,274.0,0.0,0.0,0.0,0.0,-56.93,2022-10-01 02:30:00
...,...,...,...,...,...,...,...,...,...,...
8491,2023-06-26,22:00:00,177.917,177.0,0.0,0.0,0.0,0.0,14.14,2023-06-26 22:00:00
8492,2023-06-26,22:30:00,177.938,177.0,0.0,0.0,0.0,0.0,12.93,2023-06-26 22:30:00
8493,2023-06-26,23:00:00,177.958,177.0,0.0,0.0,0.0,0.0,12.55,2023-06-26 23:00:00
8494,2023-06-26,23:30:00,177.979,177.0,0.0,0.0,0.0,0.0,11.57,2023-06-26 23:30:00
